In [ ]:
base_url = 'https://app.tandoor.dev'
token = 'abcdef123567890abc123'

In [ ]:
from bs4 import BeautifulSoup

import re
import math
import json
import pickle
import requests
import datetime


class _Bearer:
    _access_token: str
    _cookies: requests.cookies.RequestsCookieJar
    _creation_date: datetime.datetime
    _valid_until: datetime.datetime

    def __init__(self):
        self._access_token = ""
        self._request_new_token()

    def _request_new_token(self):
        answer = requests.get('https://www.hellofresh.com/recipes/')
        self._cookies = answer.cookies
        token_answer = re.search("{tokenType.*}}", answer.text).group()
        return_dict = {}
        for element in token_answer.split(','):
            values = element.split(':')
            return_dict[values[0]] = ":".join(values[1:]).strip('"')
        self._creation_date = datetime.datetime.fromisoformat(return_dict['createdAt'].rstrip("Z"))
        self._valid_until = datetime.datetime.fromtimestamp(
            self._creation_date.timestamp() + int(return_dict['expiresIn']) / 1000)
        self._access_token = return_dict['accessToken']

    def __str__(self):
        if not self._access_token:
            self._request_new_token()
        if datetime.datetime.utcnow() > self._valid_until:
            self._request_new_token()

        return self._access_token


default_bearer = _Bearer()


def request_recipes(offset=0, limit=250):
    headers = {'Authorization': f'Bearer {str(default_bearer)}'}
    payload = {"offset": offset, "limit": limit, "locale":"de-DE", "country": "de"}
    answer = requests.get("https://gw.hellofresh.com/api/recipes/search", params=payload, headers=headers)
    return answer.text

In [ ]:
max_items = 6000
max_get = 250
all_recipes = []

for i in range(math.ceil(max_items/max_get)):
    test = json.loads(request_recipes(offset=max_get*i,limit=max_get))
    all_recipes += test['items']
print(f"{len(all_recipes)} Rezepte eingesammelt.")

In [ ]:
import datetime

thermomix_recipes = {}
grouped_recipes = {}
for recipe in all_recipes:
    recipe['createdAt'] = datetime.datetime.fromisoformat(recipe['createdAt'])
    headline = f"{recipe['name']} {recipe['headline']}"
    if 'Serviervorschlag' in headline:
        continue
    if "Thermomix" in str(recipe):
        if headline in thermomix_recipes and recipe['createdAt'] < thermomix_recipes[headline]['createdAt']:
            pass
        else:
            thermomix_recipes[headline] = recipe        
    else:
        if headline in grouped_recipes and recipe['createdAt'] < grouped_recipes[headline]['createdAt']:
            pass
        else:
            grouped_recipes[headline] = recipe

print(f"{len(thermomix_recipes)} Thermomix und {len(grouped_recipes)} mehr oder weniger einzigartige Rezepte ohne Thermomix (Bei exakt identisch benamten Rezepten wird nur das neuste übernommen.)")

In [ ]:
spice_mixture = {
    # <3 https://github.com/Schischu/hello_fresh_gewuerze
    'Gewürzmischung „Hello Aloha“': 'Speisesalz, Zucker, Mangopulver (Mango, Maltodextrin), Curcuma, Rosmarin gemahlen, Kreuzkümmel, Basilikum gerebelt, Zitronenschaltengranulat (Zucker, Weizenstärke, Zitronenschalengranulat, Aromaextrakte), Chili',
    'Gewürzmischung „Hello Baharat“': 'Gewürze (Paprika, Pfeffer (schwarz), Schwarzkümmel, Koriander, Knoblauch, Chili, Piment, Macis, Anis, Kardamom, Nelke), Speisesalz',
    'Gewürzmischung „Hello Buon Appetito“': 'Tomaten, Speisesalz, Kräuter (Rosmarin, Oregano, Basilikum, Thymian), Gewürze (Zwiebeln, Chili, Knoblauch), Butterpilzpulver',
    'Gewürzmischung „HelloMediterraneo“': 'Tomaten, Speisesalz, Kräuter (Rosmarin, Oregano, Basilikum, Thymian), Gewürze (Zwiebeln, Chili, Knoblauch), Butterpilzpulver',
    'Gewürzmischung „Hello Curry“': 'Gewürze (Koriander, Curcuma, Bockshornkleesaat, Kreuzkümmel, Muskat), Paprikaextrakt',
    'Gewürzmischung „Hello Dukkah“': 'Speisesalz, Sesam, Speise-Meersalz, Kreuzkümmel, Koriander, Lorbeerblätter gemahlen, Pfeffer, Kardamom, Nelken, Muskatblüte gemahlen, Chili, Senf, Zimt',
    'Gewürzmischung „Hello Fiesta“': 'Paprika, Kumin, Knoblauch, Piment, Zimt, Chili, Meersalz, Tomatenpulver',
    'Gewürzmischung „Hello Grünzeug“': 'Süßungsmittel Erythrit (24%), Zwiebel, Thymian, Chili mild, Amchur, Salz, Kressesaat, Basilikum, Schnittlauch',
    'Gewürzmischung „Hello Harissa“': 'Chili, Paprika, Kumin, Knoblauch, Koriander, Piment, Meersalz',
    'Gewürzmischung „Hello Kokos Curry“': 'Speisesalz, Zucker, Gewürze (Bockshornklee, Chili, Kreuzkümmel, Kardamom, Nelken), 10% Kokosraspel',
    'Gewürzmischung „Hello Mezze“': 'Tomatengranulat, Knoblauchgranulat, Kreuzkümmel, Chili',
    'Gewürzmischung „Hello Muskat“': 'Speisesalz, Würze, Gewürze (Zwiebel, Lauch, Muskatnuss, Sellerie, Pastinake, Knoblauch, Kurkuma, Liebstockwurzel), Maltodextrin, Karottenpulver, Karotten, Kräuter (Petersilie)',
    'Gewürzmischung „Hello Paprika“': 'Paprika, Knoblauch, Pfeffer (schwarz), Zwiebel, Speisesalz, Toastzwiebelpulver, Tomaten, Tomatenpulver',
    'Gewürzmischung „Hello Patatas“': 'Speisesalz, Paprika, Curcuma, Kreuzkümmel, Muskat, Zwiebelgranulat',
    'Gewürzmischung „Hello Piri Piri“': 'Gewürze (Paprika, Ingwer, Pfeffer (schwarz), Koriander, Knoblauch, Zwiebel, Chili), Speisesalz, Zucker, geräucherte Paprika, Orange, Kräuter (Majoran)',
    'Gewürzmischung „Hello Shakshuka“': 'Gewürze (Paprika, Chili, Koriander, Curcuma, Kreuzkümmel), Gemüse getrocknet (Zwiebel, Knoblauch), Speisesalz, Gewürzextrakte, Schwefeldioxid',
    'Gewürzmischung „Hello Smokey“': 'Gewürze (Paprika, Pfeffer (schwarz), Senfmehl), geräucherte Paprika, Zucker, Rauchsalz (Speisesalz, Rauch), Speisesalz',
    'Gewürzmischung „Hello Smoky Paprika“': 'Speisesalz, Gewürze (Paprika, Pfeffer, Senfmehl, Chili), Zucker, Raucharoma',
    'Gewürzmischung „Hello Souflaki“': 'Knoblauch, Oregano, Tomaten, Chili mild, Cayennepfeffer',
    'Gewürzmischung „Hello Südseetraum“': 'Kalahari Wüstensalz, Vollrohrzucker (23%), Mango, Kurkuma, Rosmarin, Kreuzkümmel, Zitronenschalen, Basilikum, Chili, Fenchel',
    'Gewürzmischung „Smoky BBQ“': 'Alternative: Spicebar Smoky BBQ: https://www.spicebar.de/smoky-bbq-pfeffermischung',
    'Gewürzmischung „Gewürzmischung Desserttraum“': 'Rohrohrzucker, Vollrohrzucker, Kokosblütenzucker, Tonkabohnen, Bourbon-Vanille-Pulver',
    'Gewürzmischung „Gewürzmischung Paprikagewürz“': 'Speisesalz, Gewürze (Paprika, Pfeffer, Schwarzkümmel), Gemüse getrocknet (Zwiebeln, Knoblauch, Tomaten)',
    'Gewürzmischung „Gewürzmischung Madras Curry“': 'Kurkuma, Senf gelb, Koriandersamen, Kreuzkümmelsamen, Paprika rot edelsüß, Fenchelsamen grün, Chili mild, Ingwer, Zimt Ceylon, Zwiebelgranulat, Knoblauch',
    'Gewürzmischung „Gewürzmischung Muskat“': 'Speisesalz, Gemüse getrocknet (Lauch, Pastinaken, Karotten, Champignon, Sellerieknolle, Zwiebeln, Tomaten), Sonnenblumenöl, Maltodextrin, Gewürze (Muskat, Curcuma), Kräuter getrocknet'
}

pattern_klammer = re.compile("^.{1,}\(.*\)$")
pattern_komma = re.compile("^[^\(]{1,},.*$")

def json_recipe_converter(recipe, additional_keyword=[]):
    ingredient_dict = {}
    
    # cur_rec = None
    # while cur_rec is None or cur_rec.status_code != 200:
    #     cur_rec = requests.get(recipe['websiteUrl'])
    # soup = BeautifulSoup(cur_rec.text, 'html.parser')
    # json_rec = json.loads(soup.find(id='schema-org').string)
    
    ingredient_dict.update({el['id']: el['name'].rstrip('*') for el in recipe['ingredients']})
    
    ingredients = []
    for el in recipe['yields'][0]['ingredients']:
        name = ingredient_dict[el['id']]
        description = ""
        if pattern_klammer.fullmatch(name):
            values = name.split("(")
            name = values[0].strip()
            description = "(" + values[1].strip()
        if pattern_komma.fullmatch(name):
            values = name.split(",")
            name = values[0].strip()
            description = values[1].strip()
        if name in spice_mixture:
            description = spice_mixture[name]

        cur_ingredient = {'food': {'name': name}, 
         'unit': {'name': el['unit']},
         'amount': el['amount'], 
         'note': description}
        if not cur_ingredient['amount']:
            cur_ingredient['amount'] = 0
            cur_ingredient['unit'] = None
        ingredients += [cur_ingredient]
    working_time = 0
    if recipe['totalTime']:
        working_time = int(recipe['totalTime'].upper().replace('PT', '').replace('M',''))
    elif recipe['prepTime']:
        working_time = int(recipe['prepTime'].upper().replace('PT', '').replace('M',''))
    description = ''
    if recipe['headline'] and recipe['description']:
        description = recipe['headline'] + '\n\n' + recipe['description']
    elif recipe['headline']:
        description = recipe['description']
    elif recipe['description']:
        description = recipe['headline']

    keywords = set()
    if "Mikrowelle" in recipe['description']:
        keywords.update(['Mikrowelle'])
    keywords.update(additional_keyword)
    keywords.update([['Einfach', 'Mittel', 'Schwierig'][recipe['difficulty']-1]])
    keywords.update([cuisine['name'] for cuisine in recipe['cuisines']])
    keywords.update([tag['name'] for tag in recipe['tags']])
    keywords = [{
        "name": str(keyword).lower(),
        "icon": None,
        "description": ""
    } for keyword in keywords]
    
    nutrition = {"source": recipe['websiteUrl']}
    for nutr in recipe['nutrition']:
        #  'Energie (kJ)': '57b42a48b7e8697d4b30530d',
        if nutr['type'] == '57b42a48b7e8697d4b30530d':
            pass
        #  'Energie (kcal)': '57b42a48b7e8697d4b305304',
        elif nutr['type'] == '57b42a48b7e8697d4b305304':
            nutrition['calories'] = nutr['amount']
        # 'Fett': '57b42a48b7e8697d4b305307',
        elif nutr['type'] == '57b42a48b7e8697d4b305307':
            nutrition['fats'] = nutr['amount']
        #  'davon gesättigte Fettsäuren': '57b42a48b7e8697d4b305308',
        elif nutr['type'] == '57b42a48b7e8697d4b305308':
            pass
        #  'Kohlenhydrate': '57b42a48b7e8697d4b305305',
        elif nutr['type'] == '57b42a48b7e8697d4b305305':
            nutrition['carbohydrates'] = nutr['amount']
        #  'davon Zucker': '57b42a48b7e8697d4b305306',
        elif nutr['type'] == '57b42a48b7e8697d4b305306':
            pass
        #  'Eiweiß': '57b42a48b7e8697d4b305309',
        elif nutr['type'] == '57b42a48b7e8697d4b305309':
            nutrition['proteins'] = nutr['amount']
        #  'Salz': '57b42a48b7e8697d4b30530b'
        elif nutr['type'] == '57b42a48b7e8697d4b30530b':
            pass

    
    output_dict = {
        'name': recipe['name'],
        'description': description,
        'keywords': keywords,
        'working_time': working_time,
        "waiting_time": 0,
        "internal": True,
        'nutrition': nutrition,
        'steps':[{
            'type': 'TEXT',
            'ingredients': ingredients,
            'instruction': "\n\n\n".join([el['instructionsMarkdown'].replace("\\*", "") for el in recipe['steps']])
        }],
        'servings': 2,
        'servings_text': ""
    }
    
    image_rec = None
    while not image_rec or image_rec.status_code != 200:
        image_rec = requests.get('https://img.hellofresh.com/f_auto,fl_lossy,h_640,q_auto,w_1200/hellofresh_s3' + recipe['imagePath'])
        if image_rec.status_code == 404:
            break
    return output_dict, image_rec.content

In [ ]:
import progressbar


def import_recipes(recipe_dict, is_thermomix, base_url, token):
    failed_recipe = {}
    if is_thermomix == True:
        additional = "Thermomix"
    else:
        additional = "Ohne Thermomix"    
    for element in progressbar.progressbar(list(thermomix_recipes.values())):
        if not element['yields']:
            # recipe is old and wrong :(
            continue

        result, image = json_recipe_converter(element, ['HelloFresh', additional])

        json_request = json.dumps(result, ensure_ascii=False).encode('utf-8')

        headers = {'Authorization': f'Token {token}', 'Content-Type': 'application/json'}
        answer = requests.post(f'{base_url}/api/recipe/', headers=headers, data=json_request)
        if answer.status_code != 201:
            failed_recipe[json_request] = answer.text
            continue
        recipe_id = json.loads(answer.text)["id"]
        headers = {'Authorization': f'Token {token}'}
        answer_put = requests.put(f'{base_url}/api/recipe/{recipe_id}/image/', headers=headers, files={'image': ('thumbnail.jpg', image)})
    return failed_recipe

In [ ]:
failed_thermomix = import_recipes(thermomix_recipes, True, base_url, token)
failed_non_thermomix = import_recipes(grouped_recipes, False, base_url, token)